In [7]:
import sys
# sys.path.insert(0, '/home/gfzg/workspace/数据下载/Wind/Packages')
# sys.path.insert(0, '/home/gfzg/workspace/债券分析/Packages')
import os
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import warnings
from importlib import reload
import re
from datetime import datetime, timedelta
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm   # 这个模块也是进度条
import math
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import janitor   
# 最新版本的janitor需要找到C:\Users\username\anaconda3\Lib\site-packages路径下的janitor.py 
# 把ConfigParser改成configparse
# 把from werkzeug.contrib.fixers import ProxyFix 改成 from werkzeug.middleware.proxy_fix import ProxyFix
import time
# from progressbar import progressbar as pbar # 该用pg进度条了
from progressbar import *
pbar = ProgressBar()
from tabulate import tabulate
# 核心包
# import Wind
import Bonder 
import Selector
import dolphindb as ddb

import matplotlib as mpl
plt.rcParams["font.sans-serif"]=["SimHei"] #设置字体
plt.rcParams["axes.unicode_minus"]=False #该语句解决图像中的“-”负号的乱码问题

# 显示多行运行结果
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 运行时间比较（只到生成三表合并数据）

# DDB

In [2]:
# time.clock()默认单位为s
# 获取开始时间
start = time.perf_counter()

In [3]:

# 连接服务器节点
s = ddb.session()
isConnect = s.connect("192.168.100.43", 7988, "admin", "123456")  # 连接数据库
print(isConnect)


script = """

// 定义DDB代码的函数

// undef all   // 释放变量
// loadPlugin("/home/hanyang/luodata/PluginParquet.txt")   // 导入插件，导入一次即可

// 打印筛选条件
def slectionString(Filters){   
    print("\n数据筛选中...请稍等")
    print('筛选条件：')
    for(filter in Filters){
        if (size(filter[2]) > 1){     // isin
            print(filter[0] + ' ' + filter[1] + ' ' + concat(filter[2], ', '))
        }
        else if (isNull(filter[2])){   // isna、isnotna
            print(filter[0] + ' ' + filter[1])
        }
        else {   // "==","!=",">=","<=",">","<"
            print(filter[0] + ' ' + filter[1] + ' ' + filter[2])
        }
    }
}


// 筛选数据
def selector(df, Filters){    // 改进方向： 里面嵌一层函数

    if (!(typestr(Filters) == 'ANY VECTOR')) return df 

    data = df
    slectionString(Filters)
    selection = take(true, data.rows())
    for(filter in Filters){
        if (filter[1]=='isin'){
            selection = selection && data[filter[0]] in filter[2]
        }
        else if (filter[1] == 'isna'){
            print(filter[1])
            selection = selection && isNull(data[filter[0]])
        }
        else if (filter[1] == 'isnotna'){
            selection = selection && not isNull(data[filter[0]])
        }
        else if (filter[1] == '=='){
            selection = selection && data[filter[0]] == filter[2] && !isNull(data[filter[0]])
        }
        else if (filter[1] == '!='){
            selection = selection && data[filter[0]] != filter[2] && !isNull(data[filter[0]])
        }
        else if (filter[1] == '>='){
            selection = selection && data[filter[0]] >= filter[2] && !isNull(data[filter[0]])
        }
        else if (filter[1] == '<='){
            selection = selection && data[filter[0]] <= filter[2] && !isNull(data[filter[0]])
        }
        else if (filter[1] == '>'){
            selection = selection && data[filter[0]] > filter[2] && !isNull(data[filter[0]])
        }
        else if (filter[1] == '<'){
            selection = selection && data[filter[0]] < filter[2] && !isNull(data[filter[0]])
        }
        else {
            print('请检查逻辑符号，有效逻辑符号为["==","!=",">=","<=",">","<","isin","isna","isnotna"]')
        }
    }
    data = data[selection]
    // data = data[each(isValid, data.values()).rowAnd()]   // 删除含有缺失值的行
    return data
}


def dataCenter(StaticPath, BondsPath, FilledGuokaiPath, BondStaticFilters, BondDynamicFilters){

    // 静态筛选
    STATIC = parquet::loadParquet(StaticPath)
    STATIC.dropColumns!(`B_INFO_ISSUERCODE)   // 好像没有必要删这一列
    STATIC = selector(df = STATIC, Filters = BondStaticFilters)

    // 动态筛选
    BONDS = parquet::loadParquet(BondsPath) 
    BONDS = selector(df = BONDS, Filters = BondDynamicFilters)   

    // 动态静态结合，并计算信用利差
    df = ej(STATIC, BONDS, `S_INFO_WINDCODE) 
    df['B_ANAL_MATU_CNBD'] = round(df['B_ANAL_MATU_CNBD'], 2)  
    FILLED_GUOKAI = parquet::loadParquet(FilledGuokaiPath)
    df = lj(df, FILLED_GUOKAI, `TRADE_DT`B_ANAL_MATU_CNBD)
    df = select *, B_ANAL_YIELD_CNBD - B_ANAL_YIELD_GUOKAI as CREDIT_SPREAD from df 

    print("数据处理完成")
    return df
}


def preparePlotData(df, Filters, division_label = '', method = [0]){   // 截面日期数据未实现

    data = df.copy()
    data = data[!isNull(data[division_label])]

    if (isNull(division_label)){
        data['LABEL'] = 'ALL'
    }
    else {
        data['LABEL'] = data[division_label]
    }
    // sum(!isNull(data['LABEL']))

    data = selector(data, Filters)
    data = select * from data order by TRADE_DT, LABEL
    index = select distinct TRADE_DT, LABEL from data
    labels = select distinct LABEL from data
    dailyspread = index

    if(0 in method){
        temp = select mean(CREDIT_SPREAD) * 100 from data group by TRADE_DT, LABEL  
        dailyspread['CREDIT_SPREAD_0'] = temp[,2].values()[0] 
    }
    if(1 in method){
        temp = select median(CREDIT_SPREAD) * 100 from data group by TRADE_DT, LABEL  
        dailyspread['CREDIT_SPREAD_1'] = temp[,2].values()[0]    
    }
    if(2 in method){
        total = select TRADE_DT, LABEL, sum(B_ANAL_RESIDUALPRI) as B_ANAL_RESIDUALPRI_TOTAL from data group by TRADE_DT, LABEL
        temp = lj(data, total, `TRADE_DT`LABEL)
        update temp set CREDIT_SPREAD = B_ANAL_RESIDUALPRI / B_ANAL_RESIDUALPRI_TOTAL * CREDIT_SPREAD 
        temp = select sum(CREDIT_SPREAD) * 100 as CREDIT_SPREAD_2 from temp group by TRADE_DT, LABEL
        dailyspread['CREDIT_SPREAD_2'] = temp[,2].values()[0]     
    }
    return dailyspread
}

"""

s.run(script)   # 上传DDB函数


# 上传参数
s.upload({
    'StaticPath': "/home/hanyang/luodata/STATIC.parquet",
    'BondsPath': "/home/hanyang/luodata/BONDS.parquet",
    'FilledGuokaiPath': "/home/hanyang/luodata/FILLED_GUOKAI.parquet",
    'BondStaticFilters': [
        ('suffix', 'isin', ['IB', 'SH', 'SZ']),
        ('CATEGORY1', '==', '信用债'),
        ('isPerpetualBonds', '==', 0),
        ('IS_FAILURE', '==', 0)
    ],
    'BondDynamicFilters': [('B_ANAL_MATU_CNBD', '<=', 10)]
}  
)  

s.upload({'Filters' : [('BOND_TYPE','==','产业债')],
'division_label': 'S_INFO_COMPIND_NAME1',
'method': [0,1,2]
}
)

# 调用函数处理数据
script = """
df = dataCenter(StaticPath, BondsPath, FilledGuokaiPath, BondStaticFilters, BondDynamicFilters)
"""

s2 = "dailyspread = preparePlotData(df, Filters, division_label , method)"

s.run(script)

df = s.run("df")
# dailyspread = s.run("dailyspread")

df.info()
df.head()
# dailyspread.info()
# dailyspread.head()

True


[1790842688, 1790842784, 77557552, 1988149248, 77494144]

[77505920, 2081892848, 1593566016]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050227 entries, 0 to 1050226
Data columns (total 47 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   S_INFO_WINDCODE                 1050227 non-null  object 
 1   B_INFO_FULLNAME                 1050227 non-null  object 
 2   B_INFO_ISSUER                   1050227 non-null  object 
 3   B_INFO_CARRYDATE                1050227 non-null  object 
 4   B_INFO_MATURITYDATE             1050227 non-null  object 
 5   B_INFO_TERM_YEAR_               1050227 non-null  float64
 6   B_INFO_PAYMENTTYPE              1050227 non-null  object 
 7   S_INFO_EXCHMARKET               1050227 non-null  object 
 8   B_INFO_INTERESTFREQUENCY        1050227 non-null  object 
 9   IS_FAILURE                      1050227 non-null  float64
 10  B_INFO_GUARTYPE                 1050227 non-null  float64
 11  IS_CORPORATE_BOND               1050227 non-null  float64
 12  

,S_INFO_WINDCODE,B_INFO_FULLNAME,B_INFO_ISSUER,B_INFO_CARRYDATE,B_INFO_MATURITYDATE,B_INFO_TERM_YEAR_,B_INFO_PAYMENTTYPE,S_INFO_EXCHMARKET,B_INFO_INTERESTFREQUENCY,IS_FAILURE,B_INFO_GUARTYPE,IS_CORPORATE_BOND,B_INFO_ISSUERTYPE,B_INFO_ISSUETYPE,suffix,S_INFO_INDUSTRYNAME_WindLevel1,S_INFO_INDUSTRYNAME_WindLevel2,CATEGORY1,CATEGORY2,CATEGORY3,isPerpetualBonds,isTier2CapitalBonds,BOND_TYPE,B_AGENCY_GUARANTORNATURE,S_INFO_COMPIND_NAME1,S_INFO_COMPIND_NAME2,S_INFO_COMPIND_NAME3,S_INFO_COMPIND_NAME4,PROVINCE,CITY,DISTRICT,AREA,ADMINISTRATIVE_LEVEL,IS_SUB,__index_level_0__,TRADE_DT,B_ANAL_MATU_CNBD,B_ANAL_YIELD_CNBD,B_ANAL_RESIDUALPRI,CNBD_HIDDEN_CREDITRATING,B_INFO_CREDITRATING_BOND,B_INFO_ISSUERCODE,B_INFO_CREDITRATING_ISSUER,B_ANAL_MATU_CNBD_YEAR,data___index_level_0__,B_ANAL_YIELD_GUOKAI,CREDIT_SPREAD
0,078036.IB,2007年上海城市建设债券,上海城投(集团)有限公司,20070731,20220731,15.0,单利,NIB,每1年,0.0,507002000.0,0.0,企业,公募,IB,企业债,一般企业债,信用债,企业债,,0,0,城投债,地方国有企业,公用事业,公用事业Ⅱ,水务Ⅲ,水务,上海,上海市,,,直辖市,0.0,14987,20220104,0.57,2.6754,100.0,AAA,AAA,2002227,AAA,1.0,99017,9.87239,-7.19699
1,078036.IB,2007年上海城市建设债券,上海城投(集团)有限公司,20070731,20220731,15.0,单利,NIB,每1年,0.0,507002000.0,0.0,企业,公募,IB,企业债,一般企业债,信用债,企业债,,0,0,城投债,地方国有企业,公用事业,公用事业Ⅱ,水务Ⅲ,水务,上海,上海市,,,直辖市,0.0,14987,20220105,0.57,2.6571,100.0,AAA,AAA,2002227,AAA,1.0,99018,9.86199,-7.20489
2,078036.IB,2007年上海城市建设债券,上海城投(集团)有限公司,20070731,20220731,15.0,单利,NIB,每1年,0.0,507002000.0,0.0,企业,公募,IB,企业债,一般企业债,信用债,企业债,,0,0,城投债,地方国有企业,公用事业,公用事业Ⅱ,水务Ⅲ,水务,上海,上海市,,,直辖市,0.0,14987,20220106,0.56,2.6319,100.0,AAA,AAA,2002227,AAA,1.0,99019,9.84668,-7.21478
3,078036.IB,2007年上海城市建设债券,上海城投(集团)有限公司,20070731,20220731,15.0,单利,NIB,每1年,0.0,507002000.0,0.0,企业,公募,IB,企业债,一般企业债,信用债,企业债,,0,0,城投债,地方国有企业,公用事业,公用事业Ⅱ,水务Ⅲ,水务,上海,上海市,,,直辖市,0.0,14987,20220107,0.56,2.6493,100.0,AAA,AAA,2002227,AAA,1.0,99020,9.84458,-7.19528
4,078036.IB,2007年上海城市建设债券,上海城投(集团)有限公司,20070731,20220731,15.0,单利,NIB,每1年,0.0,507002000.0,0.0,企业,公募,IB,企业债,一般企业债,信用债,企业债,,0,0,城投债,地方国有企业,公用事业,公用事业Ⅱ,水务Ⅲ,水务,上海,上海市,,,直辖市,0.0,14987,20220110,0.55,2.6501,100.0,AAA,AAA,2002227,AAA,1.0,99021,9.84960,-7.19950


In [4]:

# 获取结束时间
end = time.perf_counter()
# 计算运行时间
runTime = end - start
runTime_ms = runTime * 1000
# 输出运行时间
print("运行时间：", runTime, "秒")
print("运行时间：", runTime_ms, "毫秒")


运行时间： 193.75397319998592 秒
运行时间： 193753.97319998592 毫秒


# Python

In [8]:
# time.clock()默认单位为s
# 获取开始时间
start = time.perf_counter()


In [9]:
# reload(Bonder)    # 重新载入之前载入的模块
# 初始化债券分析框架 Bonder
BondAnalyzer = Bonder.BondAnalyzer()

# 导入3张表
# STATIC
# BONDS
# FILLED_GUOKAI 见最上方国开债部分

# 静态筛选 
BondStaticFilters = [
    ('suffix','isin',['IB','SH','SZ']), # 默认选择标准代码
    ('CATEGORY1','==','信用债'),  # 默认选择信用债，不含利率债、同业存单、ABS、可转债
    ('isPerpetualBonds','==',0),  # 剔除：永续债
    ('IS_FAILURE','==',0)        # 剔除：发行失败
#     ('B_INFO_GUARTYPE','isna','') # 剔除：附带担保债
]
# 动态筛选
# 只考虑对应计算节点剩余期 限在 10 年以内个券
# 0. 仅保留 IB SH SZ债券
BondDynamicFilters = [
    ('B_ANAL_MATU_CNBD','<=', 10)
]
# 生成债券池
dataConfig = {
    'StaticPath': './Results/STATIC.parquet',
    'BondsPath': './Results/BONDS.parquet', 
    'FilledGuokaiPath': './Results/FILLED_GUOKAI.parquet',
    'BondStaticFilters': BondStaticFilters,
    'BondDynamicFilters': BondDynamicFilters,
}
df = BondAnalyzer.dataCenter(dataConfig)
df.info()   # 结果会多两列
df.head()

数据筛选中...请等待
筛选结果数据量： 14039
STATIC: suffix isin ['IB', 'SH', 'SZ'] and CATEGORY1 == 信用债 and isPerpetualBonds == 0 and IS_FAILURE == 0
数据筛选中...请等待
筛选结果数据量： 11014822
BONDS: B_ANAL_MATU_CNBD <= 10
信用计算利差中
数据处理完成
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050227 entries, 0 to 1050226
Data columns (total 45 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   S_INFO_WINDCODE                 1050227 non-null  object 
 1   B_INFO_FULLNAME                 1050227 non-null  object 
 2   B_INFO_ISSUER                   1050227 non-null  object 
 3   B_INFO_CARRYDATE                1050227 non-null  object 
 4   B_INFO_MATURITYDATE             1050227 non-null  object 
 5   B_INFO_TERM_YEAR_               1050227 non-null  float64
 6   B_INFO_PAYMENTTYPE              1050227 non-null  object 
 7   S_INFO_EXCHMARKET               1050227 non-null  object 
 8   B_INFO_INTERESTFREQUENCY        1050227 non-

,S_INFO_WINDCODE,B_INFO_FULLNAME,B_INFO_ISSUER,B_INFO_CARRYDATE,B_INFO_MATURITYDATE,B_INFO_TERM_YEAR_,B_INFO_PAYMENTTYPE,S_INFO_EXCHMARKET,B_INFO_INTERESTFREQUENCY,IS_FAILURE,B_INFO_GUARTYPE,IS_CORPORATE_BOND,B_INFO_ISSUERTYPE,B_INFO_ISSUETYPE,suffix,S_INFO_INDUSTRYNAME_WindLevel1,S_INFO_INDUSTRYNAME_WindLevel2,CATEGORY1,CATEGORY2,CATEGORY3,isPerpetualBonds,isTier2CapitalBonds,BOND_TYPE,B_AGENCY_GUARANTORNATURE,S_INFO_COMPIND_NAME1,S_INFO_COMPIND_NAME2,S_INFO_COMPIND_NAME3,S_INFO_COMPIND_NAME4,PROVINCE,CITY,DISTRICT,AREA,ADMINISTRATIVE_LEVEL,IS_SUB,TRADE_DT,B_ANAL_MATU_CNBD,B_ANAL_YIELD_CNBD,B_ANAL_RESIDUALPRI,CNBD_HIDDEN_CREDITRATING,B_INFO_CREDITRATING_BOND,B_INFO_ISSUERCODE,B_INFO_CREDITRATING_ISSUER,B_ANAL_MATU_CNBD_YEAR,B_ANAL_YIELD_GUOKAI,CREDIT_SPREAD
0,078036.IB,2007年上海城市建设债券,上海城投(集团)有限公司,20070731,20220731,15.0,单利,NIB,每1年,0.0,507002000.0,0.0,企业,公募,IB,企业债,一般企业债,信用债,企业债,,0,0,城投债,地方国有企业,公用事业,公用事业Ⅱ,水务Ⅲ,水务,上海,上海市,,,直辖市,0.0,20220104,0.57,2.6754,100.0,AAA,AAA,2002227,AAA,1.0,9.87239,-7.19699
1,078036.IB,2007年上海城市建设债券,上海城投(集团)有限公司,20070731,20220731,15.0,单利,NIB,每1年,0.0,507002000.0,0.0,企业,公募,IB,企业债,一般企业债,信用债,企业债,,0,0,城投债,地方国有企业,公用事业,公用事业Ⅱ,水务Ⅲ,水务,上海,上海市,,,直辖市,0.0,20220105,0.57,2.6571,100.0,AAA,AAA,2002227,AAA,1.0,9.86199,-7.20489
2,078036.IB,2007年上海城市建设债券,上海城投(集团)有限公司,20070731,20220731,15.0,单利,NIB,每1年,0.0,507002000.0,0.0,企业,公募,IB,企业债,一般企业债,信用债,企业债,,0,0,城投债,地方国有企业,公用事业,公用事业Ⅱ,水务Ⅲ,水务,上海,上海市,,,直辖市,0.0,20220106,0.56,2.6319,100.0,AAA,AAA,2002227,AAA,1.0,9.84668,-7.21478
3,078036.IB,2007年上海城市建设债券,上海城投(集团)有限公司,20070731,20220731,15.0,单利,NIB,每1年,0.0,507002000.0,0.0,企业,公募,IB,企业债,一般企业债,信用债,企业债,,0,0,城投债,地方国有企业,公用事业,公用事业Ⅱ,水务Ⅲ,水务,上海,上海市,,,直辖市,0.0,20220107,0.56,2.6493,100.0,AAA,AAA,2002227,AAA,1.0,9.84458,-7.19528
4,078036.IB,2007年上海城市建设债券,上海城投(集团)有限公司,20070731,20220731,15.0,单利,NIB,每1年,0.0,507002000.0,0.0,企业,公募,IB,企业债,一般企业债,信用债,企业债,,0,0,城投债,地方国有企业,公用事业,公用事业Ⅱ,水务Ⅲ,水务,上海,上海市,,,直辖市,0.0,20220110,0.55,2.6501,100.0,AAA,AAA,2002227,AAA,1.0,9.84960,-7.19950


In [10]:

# 获取结束时间
end = time.perf_counter()
# 计算运行时间
runTime = end - start
runTime_ms = runTime * 1000
# 输出运行时间
print("运行时间：", runTime, "秒")
print("运行时间：", runTime_ms, "毫秒")


运行时间： 11.215812099981122 秒
运行时间： 11215.812099981122 毫秒
